In [1]:
#Questions
    #What to do with unbalanced data
    #How to align test and train data from feature engineering

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2


In [2]:
from fastai.structured import *
from fastai.column_data import *
from sklearn.metrics import roc_auc_score
from pandas_summary import DataFrameSummary

np.set_printoptions(threshold = 50, edgeitems = 20)

PATH = 'data/'

In [3]:
ls {PATH}

application_test.csv                installments_payments.csv
application_test.csv.zip            installments_payments.csv.zip
application_train.csv               models/
application_train.csv.zip           POS_CASH_balance.csv
bureau_balance.csv                  POS_CASH_balance.csv.zip
bureau_balance.csv.zip              previous_application.csv
bureau.csv                          previous_application.csv.zip
bureau.csv.zip                      sample_submission.csv
credit_card_balance.csv             sample_submission.csv.zip
credit_card_balance.csv.zip         subm/
df_pref                             tmp/
HomeCredit_columns_description.csv


In [4]:
#data = pd.read_feather('tmp/data')
data = pd.read_csv(PATH + 'application_train.csv')
data.shape

(307511, 122)

In [5]:
#data_test = pd.read_feather('tmp/data_test')
data_test = pd.read_csv(PATH + 'application_test.csv')
data_test.shape

(48744, 121)

In [6]:
# nrow = 1000
#application_train = pd.read_csv(path + 'application_train.csv', nrow = 1000)
#bureau_balance = pd.read_csv(path + 'bureau_balance.csv')
#bureau = pd.read_csv(path + 'bureau.csv')
#credit_card_balance = pd.read_csv(path + 'credit_card_balance.csv')
#installments_payments = pd.read_csv(path + 'installments_payments.csv')
#pos_cash_balance = pd.read_csv(path+'POS_CASH_balance.csv')
#previous_application = pd.read_csv(path + 'previous_application.csv')

In [7]:
cat_vars = list(data.select_dtypes('object').columns); len(cat_vars)


16

In [8]:
contin_vars = list(data.select_dtypes(np.number).columns); len(contin_vars)

106

In [9]:
contin_vars.remove('SK_ID_CURR'); len(contin_vars)

105

In [10]:
contin_vars.remove('TARGET'); len(contin_vars)

104

In [11]:
dep = 'TARGET'
data = data[cat_vars+contin_vars+[dep, 'SK_ID_CURR']].copy()
# data = data[cat_vars+contin_vars].copy()
data.shape

(307511, 122)

In [12]:
dep = 'TARGET'
data_test[dep] = 0
data_test = data_test[cat_vars+contin_vars+[dep, 'SK_ID_CURR']].copy()
# data_test = data_test[cat_vars+contin_vars+[dep]].copy()
data_test.shape

(48744, 122)

In [13]:
#!pip install pandas-summary

In [14]:
def display_allRows(df):
    with pd.option_context('display.max_rows', 1000):
        with pd.option_context('display.max_columns', 10):
            display(df)

In [15]:
display_allRows(data.dtypes)

NAME_CONTRACT_TYPE               object
CODE_GENDER                      object
FLAG_OWN_CAR                     object
FLAG_OWN_REALTY                  object
NAME_TYPE_SUITE                  object
NAME_INCOME_TYPE                 object
NAME_EDUCATION_TYPE              object
NAME_FAMILY_STATUS               object
NAME_HOUSING_TYPE                object
OCCUPATION_TYPE                  object
WEEKDAY_APPR_PROCESS_START       object
ORGANIZATION_TYPE                object
FONDKAPREMONT_MODE               object
HOUSETYPE_MODE                   object
WALLSMATERIAL_MODE               object
EMERGENCYSTATE_MODE              object
CNT_CHILDREN                      int64
AMT_INCOME_TOTAL                float64
AMT_CREDIT                      float64
AMT_ANNUITY                     float64
AMT_GOODS_PRICE                 float64
REGION_POPULATION_RELATIVE      float64
DAYS_BIRTH                        int64
DAYS_EMPLOYED                     int64
DAYS_REGISTRATION               float64


In [16]:
data_test.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,...,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET,SK_ID_CURR
0,Cash loans,F,N,Y,Unaccompanied,Working,Higher education,Married,House / apartment,NaN,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,100001
1,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Low-skill Laborers,...,0,0,0.0,0.0,0.0,0.0,0.0,3.0,0,100005
2,Cash loans,M,Y,Y,NaN,Working,Higher education,Married,House / apartment,Drivers,...,0,0,0.0,0.0,0.0,0.0,1.0,4.0,0,100013
3,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Sales staff,...,0,0,0.0,0.0,0.0,0.0,0.0,3.0,0,100028
4,Cash loans,M,Y,N,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,100038


In [17]:
for v in cat_vars: data[v] = data[v].astype('category').cat.as_ordered()

In [18]:
data.shape

(307511, 122)

In [19]:
#data.select_dtypes('category')

In [20]:
#data_test.dtypes

In [21]:
#train_cats(data)

In [22]:
apply_cats(data_test, data)

In [23]:
for v in contin_vars:
    data[v] = data[v].fillna(0).astype('float32')
    data_test[v] = data_test[v].fillna(0).astype('float32')

In [24]:
data.shape, data_test.shape

((307511, 122), (48744, 122))

In [94]:
data.isnull()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,...,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET,SK_ID_CURR
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Deep Learning

Run on a sample

In [25]:
n = len(data)
idxs = get_cv_idxs(n, val_pct = 50000/n)
data_samp = data.iloc[idxs].set_index('SK_ID_CURR')
samp_size = len(data_samp); samp_size

50000

Run on full set

In [26]:
sample_size = len(data)
data_samp = data.set_index('SK_ID_CURR')
data_samp.shape

(307511, 121)

In [27]:
data_samp.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
100003,Cash loans,F,N,N,Family,State servant,Higher education,Married,House / apartment,Core staff,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
100004,Revolving loans,M,Y,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
100006,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
100007,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


ModelData object

In [28]:
df, y, nas, mapper = proc_df(data_samp, 'TARGET', do_scale=True)

In [29]:
dfs = DataFrameSummary(df)

In [31]:
display_allRows(dfs.summary().transpose())

,count,mean,std,min,25%,...,counts,uniques,missing,missing_perc,types
NAME_CONTRACT_TYPE,307511,1.09521,0.293509,1,1,...,307511,2,0,0%,bool
CODE_GENDER,307511,1.34167,0.474297,1,1,...,307511,3,0,0%,numeric
FLAG_OWN_CAR,307511,1.34011,0.473746,1,1,...,307511,2,0,0%,bool
FLAG_OWN_REALTY,307511,1.69367,0.460968,1,1,...,307511,2,0,0%,bool
NAME_TYPE_SUITE,307511,6.19358,1.81701,0,7,...,307511,8,0,0%,numeric
NAME_INCOME_TYPE,307511,5.67029,2.54452,1,4,...,307511,8,0,0%,numeric
NAME_EDUCATION_TYPE,307511,4.18827,1.29875,1,3,...,307511,5,0,0%,numeric
NAME_FAMILY_STATUS,307511,2.47231,1.16888,1,2,...,307511,6,0,0%,numeric
NAME_HOUSING_TYPE,307511,2.29039,0.951168,1,2,...,307511,6,0,0%,numeric
OCCUPATION_TYPE,307511,6.07636,5.49102,0,0,...,307511,19,0,0%,numeric


In [27]:
#data_test['TARGET'] = 0 #??

In [32]:
df_test, _, nas, mapper = proc_df(data_test, 'TARGET', do_scale=True, 
                    skip_flds=['SK_ID_CURR'], mapper=mapper, na_dict=nas)

In [33]:
#validation index
train_ratio = 0.75
train_size = int(samp_size * train_ratio); train_size
val_idx = list(range(train_size, len(df))); len(val_idx)

270011

In [34]:
#df.dropna(inplace=True)

In [35]:
#ModelData object
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, y, 
                                       cat_flds = cat_vars, bs=128, test_df=df_test )

In [36]:
df.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,2,1,2,7,8,5,4,2,9,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155838,-0.269947,-0.30862,-0.346719
100003,1,1,1,1,2,5,2,2,2,4,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155838,-0.269947,-0.30862,-0.885565
100004,2,2,2,2,7,8,5,4,2,9,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155838,-0.269947,-0.30862,-0.885565
100006,1,1,1,2,7,8,5,1,2,9,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155838,-0.269947,-0.30862,-0.885565
100007,1,2,1,2,7,8,5,4,2,4,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155838,-0.269947,-0.30862,-0.885565


In [44]:
display_allRows(df.isnull().transpose())

SK_ID_CURR,384575,214010,142232,389171,283617,...,392541,315401,319459,145509,206854
NAME_CONTRACT_TYPE,False,False,False,False,False,...,False,False,False,False,False
CODE_GENDER,False,False,False,False,False,...,False,False,False,False,False
FLAG_OWN_CAR,False,False,False,False,False,...,False,False,False,False,False
FLAG_OWN_REALTY,False,False,False,False,False,...,False,False,False,False,False
NAME_TYPE_SUITE,False,False,False,False,False,...,False,False,False,False,False
NAME_INCOME_TYPE,False,False,False,False,False,...,False,False,False,False,False
NAME_EDUCATION_TYPE,False,False,False,False,False,...,False,False,False,False,False
NAME_FAMILY_STATUS,False,False,False,False,False,...,False,False,False,False,False
NAME_HOUSING_TYPE,False,False,False,False,False,...,False,False,False,False,False
OCCUPATION_TYPE,False,False,False,False,False,...,False,False,False,False,False


In [37]:
#show cardinality for each categories
cat_sz = [(c, len(data_samp[c].cat.categories)+1) for c in cat_vars]

In [38]:
cat_sz

[('NAME_CONTRACT_TYPE', 3),
 ('CODE_GENDER', 4),
 ('FLAG_OWN_CAR', 3),
 ('FLAG_OWN_REALTY', 3),
 ('NAME_TYPE_SUITE', 8),
 ('NAME_INCOME_TYPE', 9),
 ('NAME_EDUCATION_TYPE', 6),
 ('NAME_FAMILY_STATUS', 7),
 ('NAME_HOUSING_TYPE', 7),
 ('OCCUPATION_TYPE', 19),
 ('WEEKDAY_APPR_PROCESS_START', 8),
 ('ORGANIZATION_TYPE', 59),
 ('FONDKAPREMONT_MODE', 5),
 ('HOUSETYPE_MODE', 4),
 ('WALLSMATERIAL_MODE', 8),
 ('EMERGENCYSTATE_MODE', 3)]

In [39]:
#set embeding sizes
emb_szs = [(c, min(50,(c+1)//2)) for _, c in cat_sz]

In [40]:
emb_szs

[(3, 2),
 (4, 2),
 (3, 2),
 (3, 2),
 (8, 4),
 (9, 5),
 (6, 3),
 (7, 4),
 (7, 4),
 (19, 10),
 (8, 4),
 (59, 30),
 (5, 3),
 (4, 2),
 (8, 4),
 (3, 2)]

In [41]:
m = md.get_learner(emb_szs, len(data.columns)-len(cat_vars),
                  0.04, 1, [1000, 500], [0.001, 0.01])
lr = 1e-3

In [42]:
len(data.columns)-len(cat_vars)

106

In [43]:
m.lr_find()

  0%|          | 0/293 [00:00<?, ?it/s]


RuntimeError: running_mean should contain 104 elements not 106

In [42]:
m.fit(lr, 3)

  0%|          | 0/293 [00:00<?, ?it/s]


RuntimeError: running_mean should contain 104 elements not 106